In [247]:
import os 

In [248]:
%pwd

'/'

In [249]:
os.chdir('../') # back one folder 

In [250]:
%pwd

'/'

In [251]:
from dataclasses import dataclass
from pathlib import Path

In [252]:
@dataclass(frozen=True)
class DataIngestionConfig:
    # Return these types 
    root_dir : Path
    source_url : str 
    local_data_file : Path
    unzip_file : Path

In [253]:
# Read the yaml file 
from srcTrackVision.constants import CONFIG_FILE_PATH , PARMAS_FILE_PATH
from srcTrackVision.utils.common import read_yaml ,  create_directories


In [254]:
type(str(CONFIG_FILE_PATH))

str

In [255]:
class ConfigurationManager:
    def __init__(
            self , 
            config_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml',
            params_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml' ):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # Create the root dir 
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file = config.local_data_file,
            unzip_file = config.unzip_file 

        )

    
        return data_ingestion_config
         

In [271]:
import os 
from urllib.request import urlopen
import zipfile 
from srcTrackVision import logger
from srcTrackVision.utils.common import get_size

In [278]:
class DataIngestion:
    def __init__(self , config : DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if (not os.path.exists(self.config.local_data_file)):
            fin = urlopen(self.config.source_url)
            data = fin.read()
            with open(self.config.local_data_file, 'wb') as f:
                f.write(data)
            logger.info(f"{self.config.local_data_file} downloaded!")
        else : 
            logger.info(f'File already exists of size : {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_file
        os.makedirs(unzip_path , exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:

            f.extractall(unzip_path)

In [279]:
# Create the pipeline
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 
    

2023-06-17 12:28:30,858 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml loaded successfully
2023-06-17 12:28:30,860 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml loaded successfully
2023-06-17 12:28:30,861 - trackvisionlogger - INFO - created directory at :/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts
2023-06-17 12:28:30,862 - trackvisionlogger - INFO - created directory at :/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/data_ingestion
2023-06-17 12:28:30,863 - trackvisionlogger - INFO - File already exists of size : ~4602 KB
